<a href="https://colab.research.google.com/github/aimanyounises1/NLP_WEB/blob/master/AraBERT_NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#installing dependencies

In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))
    !nvidia-smi

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB
Sat Jan 30 08:48:56 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    27W / 250W |     10MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                

In [ ]:
!pip install optuna
!pip install seqeval
!pip install sentencepiece
!git clone https://github.com/huggingface/transformers
!cd transformers && pip install .
!pip install farasapy
!pip install pyarabic
!git clone https://github.com/aub-mind/arabert

     |████████████████████████████████| 286kB 8.0MB/s 
     |████████████████████████████████| 81kB 9.0MB/s 
     |████████████████████████████████| 1.1MB 10.2MB/s 
     |████████████████████████████████| 133kB 35.1MB/s 
     |████████████████████████████████| 51kB 8.5MB/s 
     |████████████████████████████████| 112kB 32.4MB/s 
     |████████████████████████████████| 481kB 25.1MB/s 
  Created wheel for alembic: filename=alembic-1.5.3-py2.py3-none-any.whl size=155547 sha256=3536a657b1903e5ac2b0379e74db8000f9d974172c2dc8a6a426326f4423f265
  Stored in directory: /root/.cache/pip/wheels/7f/ed/1b/a66a0cbca75fd3e374bd5cc60c443e5675f10ef1f1f78ec31d
  Created wheel for PrettyTable: filename=prettytable-0.7.2-cp36-none-any.whl size=13702 sha256=ccdabed2799bd46cdf95369426dde661cf13a1bd5883fb1e85129db00cfb24c4
  Stored in directory: /root/.cache/pip/wheels/80/34/1c/3967380d9676d162cb59513bd9dc862d0584e045a162095606
  Created wheel for Mako: filename=Mako-1.1.4-py2.py3-none-any.whl size=75675 sha

#Creating training datasets

In [ ]:
import pandas as pd
import numpy as np

from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import train_test_split
all_datasets= []

In [ ]:
class Dataset:
    def __init__(
        self,
        name,
        train,
        test,
        label_list,
    ):
        self.name = name
        self.train = train
        self.test = test
        self.label_list = label_list

all_datasets = []

#ANERCorp

We are using the ANERCorp with the Camel Lab splits from https://camel.abudhabi.nyu.edu/anercorp/

In [ ]:
!unzip "/content/drive/MyDrive/ANERcorp-CamelLabSplits.zip"

Archive:  /content/drive/MyDrive/ANERcorp-CamelLabSplits.zip


In [ ]:
from collections import Counter
import matplotlib.pyplot as plt

`read_ANERcorp` return a list of examples

each example has a list of tokens and a list of corresponding labels

In [ ]:
def read_ANERcorp(path):
  with open(path,'r',encoding='utf-8') as f:
    data = []
    sentence = []
    label = []
    for line in f:
      if line=='\n':
        if len(sentence) > 0:
          data.append((sentence,label))
          sentence = []
          label = []
        continue
      splits = line.split()
      sentence.append(splits[0])
      label.append(splits[1])
    if len(sentence) > 0:
      data.append((sentence,label))
  return data

In [ ]:
ANERCorp_path = './ANERcorp-CamelLabSplits/'
data_train = read_ANERcorp(ANERCorp_path+'ANERCorp_CamelLab_train.txt')
data_test = read_ANERcorp(ANERCorp_path+'ANERCorp_CamelLab_test.txt')

print(Counter([ label for sentence in data_test for label in sentence[1]]))
print(Counter([ label for sentence in data_train for label in sentence[1]]))

label_list = list(Counter([ label for sentence in data_test for label in sentence[1]]).keys())
print(label_list)

In [ ]:
print("Training Sentence Lengths: ")
plt.hist([ len(sentence[0]) for sentence in data_train],bins=range(0,256,2))
plt.show()
print(sum([len(sentence[0]) > 512 for sentence in data_train]))

print("Testing Sentence Lengths: ")
plt.hist([ len(sentence[0]) for sentence in data_test],bins=range(0,256,2))
plt.show()
print(sum([len(sentence[0]) > 256 for sentence in data_test]))

256 seems a good choice for max_length

In [ ]:
data_AJGT = Dataset("ANERCorp", data_train, data_test, label_list)
all_datasets.append(data_AJGT)

#Trainer

In [ ]:
from arabert.preprocess import ArabertPreprocessor
import numpy as np
from seqeval.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report

from transformers import AutoConfig, AutoModelForTokenClassification, AutoTokenizer
from transformers import Trainer , TrainingArguments
from transformers.trainer_utils import EvaluationStrategy
from transformers.data.processors.utils import InputFeatures
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from sklearn.utils import resample
import logging
import torch
import optuna 

In [ ]:
logging.basicConfig(level=logging.WARNING)
logger = logging.getLogger(__name__)

In [ ]:
for x in all_datasets:
  print(x.name)

In [ ]:
dataset_name = 'ANERCorp'
model_name = 'aubmindlab/bert-base-arabertv02'
task_name = 'tokenclassification'

In [ ]:
for d in all_datasets:
  if d.name==dataset_name:
    selected_dataset = d
    print('Dataset found')
    break

Dataset found


Create, preprocess, and tokenize ANERCorp

In [ ]:
class NERDataset:
  def __init__(self, texts, tags, label_list, model_name, max_length):
    self.texts = texts
    self.tags = tags
    self.label_map = {label: i for i, label in enumerate(label_list)}
    self.preprocessor = ArabertPreprocessor(model_name.split("/")[-1])    
    self.pad_token_label_id = torch.nn.CrossEntropyLoss().ignore_index
    # Use cross entropy ignore_index as padding label id so that only
    # real label ids contribute to the loss later.
    self.tokenizer = AutoTokenizer.from_pretrained(model_name)
    self.max_length = max_length

     
  def __len__(self):
    return len(self.texts)
  
  def __getitem__(self, item):
    textlist = self.texts[item]
    tags = self.tags[item]

    tokens = []
    label_ids = []
    for word, label in zip(textlist, tags):      
      clean_word = self.preprocessor.preprocess(word)  
      word_tokens = self.tokenizer.tokenize(clean_word)

      if len(word_tokens) > 0:
        tokens.extend(word_tokens)    
        # Use the real label id for the first token of the word, and padding ids for the remaining tokens
        label_ids.extend([self.label_map[label]] + [self.pad_token_label_id] * (len(word_tokens) - 1))
 
    # Account for [CLS] and [SEP] with "- 2" and with "- 3" for RoBERTa.
    special_tokens_count = self.tokenizer.num_special_tokens_to_add()
    if len(tokens) > self.max_length - special_tokens_count:
      tokens = tokens[: (self.max_length - special_tokens_count)]
      label_ids = label_ids[: (self.max_length - special_tokens_count)]
  
    #Add the [SEP] token
    tokens += [self.tokenizer.sep_token]
    label_ids += [self.pad_token_label_id]
    token_type_ids = [0] * len(tokens)

    #Add the [CLS] TOKEN
    tokens = [self.tokenizer.cls_token] + tokens
    label_ids = [self.pad_token_label_id] + label_ids
    token_type_ids = [0] + token_type_ids

    input_ids = self.tokenizer.convert_tokens_to_ids(tokens)

    # The mask has 1 for real tokens and 0 for padding tokens. Only real
    # tokens are attended to.
    attention_mask = [1] * len(input_ids)

    # Zero-pad up to the sequence length.
    padding_length = self.max_length - len(input_ids)

    input_ids += [self.tokenizer.pad_token_id] * padding_length
    attention_mask += [0] * padding_length
    token_type_ids += [0] * padding_length
    label_ids += [self.pad_token_label_id] * padding_length

    assert len(input_ids) == self.max_length
    assert len(attention_mask) == self.max_length
    assert len(token_type_ids) == self.max_length
    assert len(label_ids) == self.max_length

    # if item < 5:
    #   print("*** Example ***")
    #   print("tokens:", " ".join([str(x) for x in tokens]))
    #   print("input_ids:", " ".join([str(x) for x in input_ids]))
    #   print("attention_mask:", " ".join([str(x) for x in attention_mask]))
    #   print("token_type_ids:", " ".join([str(x) for x in token_type_ids]))
    #   print("label_ids:", " ".join([str(x) for x in label_ids]))
    
    return {
        'input_ids' : torch.tensor(input_ids, dtype=torch.long),
        'attention_mask' : torch.tensor(attention_mask, dtype=torch.long),
        'token_type_ids' : torch.tensor(token_type_ids, dtype=torch.long),
        'labels' : torch.tensor(label_ids, dtype=torch.long)       
    }

In [ ]:
label_map = { v:index for index, v in enumerate(selected_dataset.label_list) }
print(label_map)

train_dataset = NERDataset(
    texts=[x[0] for x in selected_dataset.train],
    tags=[x[1] for x in selected_dataset.train],
    label_list=selected_dataset.label_list,
    model_name=model_name,
    max_length=256
    )

test_dataset = NERDataset(
    texts=[x[0] for x in selected_dataset.test],
    tags=[x[1] for x in selected_dataset.test],
    label_list=selected_dataset.label_list,
    model_name=model_name,
    max_length=256
    )

{'B-LOC': 0, 'O': 1, 'B-PERS': 2, 'I-PERS': 3, 'B-ORG': 4, 'I-LOC': 5, 'I-ORG': 6, 'B-MISC': 7, 'I-MISC': 8}


Get NER Model

In [ ]:
def model_init():
    return AutoModelForTokenClassification.from_pretrained(model_name, return_dict=True, num_labels=len(label_map))

In [ ]:
inv_label_map = {i: label for i, label in enumerate(label_list)}

def align_predictions(predictions, label_ids):
    preds = np.argmax(predictions, axis=2)

    batch_size, seq_len = preds.shape

    out_label_list = [[] for _ in range(batch_size)]
    preds_list = [[] for _ in range(batch_size)]

    for i in range(batch_size):
        for j in range(seq_len):
            if label_ids[i, j] != torch.nn.CrossEntropyLoss().ignore_index:
                out_label_list[i].append(inv_label_map[label_ids[i][j]])
                preds_list[i].append(inv_label_map[preds[i][j]])

    return preds_list, out_label_list

def compute_metrics(p):
    preds_list, out_label_list = align_predictions(p.predictions,p.label_ids)
    #print(classification_report(out_label_list, preds_list,digits=4))
    return {
        "accuracy_score": accuracy_score(out_label_list, preds_list),
        "precision": precision_score(out_label_list, preds_list),
        "recall": recall_score(out_label_list, preds_list),
        "f1": f1_score(out_label_list, preds_list),
    }

# Doing Hyper parameter search with GRIDsearch sampler i.e. optuna will quit when all combination of values in the search space have been tested

In [ ]:
training_args = TrainingArguments("./train")
training_args.evaluate_during_training = True
training_args.adam_epsilon = 1e-8
training_args.fp16 = True
training_args.per_device_train_batch_size = 4
training_args.per_device_eval_batch_size = 16
training_args.gradient_accumulation_steps = 8
training_args.num_train_epochs= 6
training_args.evaluation_strategy = EvaluationStrategy.EPOCH
training_args.save_steps = 100000
training_args.disable_tqdm = True
training_args.lr_scheduler_type = 'cosine'

In [ ]:
steps_per_epoch = (len(selected_dataset.train)// (training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps))
total_steps = steps_per_epoch * training_args.num_train_epochs
print(steps_per_epoch)
print(total_steps)

124
744


In [ ]:
trainer = Trainer(
    args=training_args,
    train_dataset=train_dataset, 
    eval_dataset=test_dataset, 
    model_init=model_init,
    compute_metrics=compute_metrics,
)

Some weights of the model checkpoint at aubmindlab/bert-large-arabertv02 were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initi

In [ ]:
def my_hp_space(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 2e-5, 7e-5, step=1e-5),
        "seed": trial.suggest_categorical("seed", [0, 1, 42, 666, 123, 12345]),
        "warmup_steps": trial.suggest_int("warmup_steps",0,total_steps*0.1,step=total_steps*0.1*0.5)
    }

search_space = {
    "learning_rate":  list(np.arange(2e-5, 7e-5, 1e-5)),
    "seed":  [0, 1, 42, 666, 123, 12345],
    "warmup_steps": list(range(0, int((total_steps)*0.1)+1, int(total_steps*0.1*0.5)))
}
search_space

{'learning_rate': [2e-05,
  3.0000000000000004e-05,
  4.000000000000001e-05,
  5.000000000000001e-05,
  6.000000000000001e-05],
 'seed': [0, 1, 42, 666, 123, 12345],
 'warmup_steps': [0, 37, 74]}

In [ ]:
def my_objective(metrics):
    return metrics['eval_f1']

In [ ]:
name = "NER-arabert-large-v02"

In [ ]:
best_run = trainer.hyperparameter_search(direction="maximize",
                                         hp_space=my_hp_space,
                                         compute_objective=my_objective,
                                         n_trials=None,
                                         pruner=optuna.pruners.NopPruner(),
                                         sampler=optuna.samplers.GridSampler(search_space),
                                         study_name="NER-arabert-large-v02",
                                         storage="sqlite:////content/drive/MyDrive/optuna_runs/{}.db".format(name),
                                         load_if_exists=False
                                         )

[I 2021-01-06 19:52:08,426] A new study created in RDB with name: NER-arabert-large-v02
Some weights of the model checkpoint at aubmindlab/bert-large-arabertv02 were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertFor

{'eval_loss': 0.14968341588974, 'eval_accuracy_score': 0.9664003853719241, 'eval_precision': 0.802754331408263, 'eval_recall': 0.7915024091108191, 'eval_f1': 0.7970886634318484, 'eval_runtime': 12.5556, 'eval_samples_per_second': 73.673, 'epoch': 0.9979879275653923}
{'eval_loss': 0.14608950912952423, 'eval_accuracy_score': 0.9681666733571515, 'eval_precision': 0.8205709107385591, 'eval_recall': 0.7932544897065265, 'eval_f1': 0.8066815144766147, 'eval_runtime': 12.4532, 'eval_samples_per_second': 74.278, 'epoch': 1.9979879275653922}
{'eval_loss': 0.1497354358434677, 'eval_accuracy_score': 0.9700533900686443, 'eval_precision': 0.8355173971983733, 'eval_recall': 0.8098992553657468, 'eval_f1': 0.822508896797153, 'eval_runtime': 12.4806, 'eval_samples_per_second': 74.115, 'epoch': 2.9979879275653922}
{'eval_loss': 0.15973614156246185, 'eval_accuracy_score': 0.970213961703665, 'eval_precision': 0.835820895522388, 'eval_recall': 0.80946123521682, 'eval_f1': 0.822429906542056, 'eval_runtime': 

[I 2021-01-06 20:05:49,234] Trial 0 finished with value: 0.82018226272505 and parameters: {'learning_rate': 6.000000000000001e-05, 'seed': 123, 'warmup_steps': 0}. Best is trial 0 with value: 0.82018226272505.


{'eval_loss': 0.16720832884311676, 'eval_accuracy_score': 0.9703343904299305, 'eval_precision': 0.8325812274368231, 'eval_recall': 0.8081471747700394, 'eval_f1': 0.82018226272505, 'eval_runtime': 12.3937, 'eval_samples_per_second': 74.635, 'epoch': 5.997987927565393}
{'train_runtime': 795.8278, 'train_samples_per_second': 0.935, 'epoch': 5.997987927565393}


Some weights of the model checkpoint at aubmindlab/bert-large-arabertv02 were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initi

...

{'eval_loss': 0.14718475937843323, 'eval_accuracy_score': 0.9711372486050339, 'eval_precision': 0.8369079535299374, 'eval_recall': 0.8204117389399912, 'eval_f1': 0.8285777482857773, 'eval_runtime': 12.5179, 'eval_samples_per_second': 73.894, 'epoch': 5.997987927565393}
{'train_runtime': 793.9273, 'train_samples_per_second': 0.937, 'epoch': 5.997987927565393}


Some weights of the model checkpoint at aubmindlab/bert-large-arabertv02 were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initi

{'eval_loss': 0.1570618599653244, 'eval_accuracy_score': 0.9633896672152864, 'eval_precision': 0.8187762727697337, 'eval_recall': 0.7678493210687691, 'eval_f1': 0.7924954792043399, 'eval_runtime': 12.4615, 'eval_samples_per_second': 74.229, 'epoch': 0.9979879275653923}
{'eval_loss': 0.1403229534626007, 'eval_accuracy_score': 0.9693709606198065, 'eval_precision': 0.8144375553587245, 'eval_recall': 0.8055190538764783, 'eval_f1': 0.8099537546795859, 'eval_runtime': 12.5189, 'eval_samples_per_second': 73.889, 'epoch': 1.9979879275653922}
{'eval_loss': 0.14377336204051971, 'eval_accuracy_score': 0.9696118180723375, 'eval_precision': 0.8271549799017418, 'eval_recall': 0.8112133158125274, 'eval_f1': 0.8191065900044228, 'eval_runtime': 12.4886, 'eval_samples_per_second': 74.067, 'epoch': 2.9979879275653922}
{'eval_loss': 0.14475049078464508, 'eval_accuracy_score': 0.9709766769700132, 'eval_precision': 0.8302895322939866, 'eval_recall': 0.8164695575996496, 'eval_f1': 0.823321554770318, 'eval_ru

[I 2021-01-06 22:21:39,242] Trial 10 finished with value: 0.8267140004437542 and parameters: {'learning_rate': 3.0000000000000004e-05, 'seed': 0, 'warmup_steps': 37}. Best is trial 5 with value: 0.8326693227091634.


{'eval_loss': 0.15130630135536194, 'eval_accuracy_score': 0.9703343904299305, 'eval_precision': 0.8376798561151079, 'eval_recall': 0.8160315374507228, 'eval_f1': 0.8267140004437542, 'eval_runtime': 12.4236, 'eval_samples_per_second': 74.455, 'epoch': 5.997987927565393}
{'train_runtime': 795.1332, 'train_samples_per_second': 0.936, 'epoch': 5.997987927565393}


In [ ]:
best_run

#Regular Training

In [ ]:
training_args = TrainingArguments("./train")
training_args.evaluate_during_training = True
training_args.adam_epsilon = 1e-8
training_args.learning_rate = 5e-5
training_args.fp16 = True
training_args.per_device_train_batch_size = 16
training_args.per_device_eval_batch_size = 16
training_args.gradient_accumulation_steps = 2
training_args.num_train_epochs= 8


steps_per_epoch = (len(selected_dataset.train)// (training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps))
total_steps = steps_per_epoch * training_args.num_train_epochs
print(steps_per_epoch)
print(total_steps)
#Warmup_ratio
warmup_ratio = 0.1
training_args.warmup_steps = total_steps*warmup_ratio

training_args.evaluation_strategy = EvaluationStrategy.EPOCH
# training_args.logging_steps = 200
training_args.save_steps = 100000 #don't want to save any model
training_args.seed = 42
training_args.disable_tqdm = False
training_args.lr_scheduler_type = 'cosine'

In [ ]:
trainer = Trainer(
    model = model_init(),
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

Some weights of the model checkpoint at aubmindlab/bert-base-arabertv02 were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initia

In [ ]:
trainer.train()

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning:

Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate



Epoch,Training Loss,Validation Loss,Accuracy Score,Precision,Recall,F1,Runtime,Samples Per Second
0,No log,0.161440,0.962306,0.820141,0.766973,0.792666,8.584500,107.752000
1,No log,0.145242,0.966962,0.830258,0.788436,0.808807,8.437500,109.629000
2,No log,0.148088,0.969732,0.834677,0.816032,0.825249,8.470400,109.204000
3,No log,0.144629,0.971499,0.840444,0.828296,0.834326,8.426100,109.779000
4,0.111123,0.153060,0.970776,0.845632,0.818222,0.831701,8.732600,105.925000
5,0.111123,0.159308,0.970896,0.844485,0.818222,0.831146,8.544100,108.262000
6,0.111123,0.160349,0.971097,0.842318,0.821288,0.831670,8.606100,107.483000
7,0.111123,0.160672,0.971057,0.842484,0.819974,0.831077,8.430300,109.723000


TrainOutput(global_step=992, training_loss=0.06601395337812362, metrics={'train_runtime': 376.3862, 'train_samples_per_second': 2.636, 'total_flos': 6570640586138112, 'epoch': 7.995983935742972})

In [ ]:
trainer.save_model("SOME_PATH")